## Resolve dependências

In [1]:
!pip install -q albumentations==1.4.6 opencv-python pycocotools tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q pycocotools albumentations==1.4.6 opencv-python tqdm
!pip uninstall -y sympy
!pip install sympy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 13.7 MB/s eta 0:00:00
Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 123.4 MB/s eta 0:00:00


In [2]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.24.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 694356, done.
remote: Counting objects: 100% (1691/1691), done.
remote: Compressing objects: 100% (879/879), done.
remote: Total 694356 (delta 1519), reused 885 (delta 804), pack-reused 692665 (from 3)
Receiving objects: 100% (694356/694356), 1.22 GiB | 16.94 MiB/s, done.
Resolving deltas: 100% (650252/650252), done.
fatal: not a git repository (or any of the parent directories): .git


In [3]:
import sys
# sys.path.append("/content/drive/MyDrive/TCC/src/")
sys.path.append("/content/drive/MyDrive/TCC/notebooks/Faster R-CNN")

In [4]:
checkpoint_path = '/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/pools/model/finetuning_c_23.pth'

In [5]:
from pathlib import Path

DATASET_ROOT = Path("/content/drive/MyDrive/TCC/DATASET")
paths = {
    "bh_pools": {
        "images": {
            "train": DATASET_ROOT/"bh_pools/images/train",
            "val":   DATASET_ROOT/"bh_pools/images/val",
            "test":  DATASET_ROOT/"bh_pools/images/test",
        },
        "ann": {
            "train": DATASET_ROOT/"bh_pools/annotations/instances_train.json",
            "val":   DATASET_ROOT/"bh_pools/annotations/instances_val.json",
            "test":  DATASET_ROOT/"bh_pools/annotations/instances_test.json",
        }
    }
}

In [6]:
import sys
from pathlib import Path
sys.path.append("/content/drive/MyDrive/TCC/src/")
DATASET_ROOT = Path("/content/drive/MyDrive/TCC/campinas_pool_dataset")

campinas_paths = {
    "bh_watertanks": {
        "images": {
            "train": DATASET_ROOT/"train/images/",
            "test":  DATASET_ROOT/"test/images/",
        },
        "ann": {
            "train": DATASET_ROOT/"train/annotations/instances_train.json",
            "test":  DATASET_ROOT/"test/annotations/instances_test.json",
        }
    }
}

## Carrega Loaders

In [7]:
from my_utils.coco_dataset import CocoDetDataset
from my_utils.my_utils import get_transforms, collate_fn, evaluate_and_visualize
from torch.utils.data import Dataset, DataLoader


pools_train = CocoDetDataset(
    paths["bh_pools"]["images"]["train"],
    paths["bh_pools"]["ann"]["train"],
    transforms=get_transforms(train=True))

pools_val   = CocoDetDataset(
    paths["bh_pools"]["images"]["val"],
    paths["bh_pools"]["ann"]["val"],
    transforms=get_transforms(train=False))

pools_test  = CocoDetDataset(
    paths["bh_pools"]["images"]["test"],
    paths["bh_pools"]["ann"]["test"],
    transforms=get_transforms(train=False))

pools_train_loader = DataLoader(pools_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
pools_val_loader   = DataLoader(pools_val,   batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
pools_test_loader  = DataLoader(pools_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

campinas_pools_train = CocoDetDataset(
    campinas_paths["bh_watertanks"]["images"]["train"],
    campinas_paths["bh_watertanks"]["ann"]["train"],
    transforms=get_transforms(train=True))

campinas_pools_test  = CocoDetDataset(
    campinas_paths["bh_watertanks"]["images"]["test"],
    campinas_paths["bh_watertanks"]["ann"]["test"],
    transforms=get_transforms(train=False))


campinas_pools_train_loader = DataLoader(campinas_pools_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
campinas_pools_test_loader  = DataLoader(campinas_pools_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

print("campinas-pools:", len(campinas_pools_train), len(campinas_pools_test))

loading annotations into memory...
Done (t=2.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
loading annotations into memory...
Done (t=0.89s)
creating index...
index created!
loading annotations into memory...
Done (t=2.09s)
creating index...
index created!
loading annotations into memory...
Done (t=1.47s)
creating index...
index created!
campinas-pools: 52 22


## Inicializa model

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class WindowMHSA(nn.Module):
    def __init__(self, dim, num_heads=4, window_size=7, dim_reduction=None, dropout=0.0):
        super().__init__()
        self.window_size = window_size
        self.in_dim = dim
        self.mid_dim = dim_reduction or dim  # opcional reduzir canais
        self.pre = nn.Conv2d(dim, self.mid_dim, 1, bias=False) if self.mid_dim != dim else nn.Identity()
        self.norm1 = nn.LayerNorm(self.mid_dim)
        self.attn = nn.MultiheadAttention(self.mid_dim, num_heads, dropout=dropout, batch_first=True)
        # FFN
        mlp_hidden = int(self.mid_dim * 4)
        self.mlp = nn.Sequential(
            nn.Linear(self.mid_dim, mlp_hidden),
            nn.GELU(),
            nn.Linear(mlp_hidden, self.mid_dim),
        )
        self.norm2 = nn.LayerNorm(self.mid_dim)
        self.post = nn.Conv2d(self.mid_dim, dim, 1, bias=False) if self.mid_dim != dim else nn.Identity()

    def _pad_to_window(self, x):
        B, C, H, W = x.shape
        ws = self.window_size
        padH = (ws - H % ws) % ws
        padW = (ws - W % ws) % ws
        if padH or padW:
            x = F.pad(x, (0, padW, 0, padH))
        return x, H, W, padH, padW

    def forward(self, x):  # x: [B, C, H, W]
        B, C, H, W = x.shape
        y = self.pre(x)
        y, oH, oW, padH, padW = self._pad_to_window(y)
        B, C2, H2, W2 = y.shape
        ws = self.window_size

        # [B, C2, H2, W2] -> janelas: [B*nw, ws*ws, C2]
        y = y.unfold(2, ws, ws).unfold(3, ws, ws)  # [B, C2, nH, nW, ws, ws]
        nH, nW = y.shape[2], y.shape[3]
        y = y.permute(0,2,3,1,4,5).contiguous().view(B*nH*nW, C2, ws*ws)
        y = y.permute(0,2,1).contiguous()  # [B*nw, L, C2]

        # MHSA por janela
        y = self.norm1(y)
        attn_out, _ = self.attn(y, y, y)  # [B*nw, L, C2]
        y = y + attn_out
        y = y + self.mlp(self.norm2(y))

        # volta para [B, C2, H2, W2]
        y = y.permute(0,2,1).contiguous().view(B, nH, nW, C2, ws, ws)
        y = y.permute(0,3,1,4,2,5).contiguous().view(B, C2, nH*ws, nW*ws)
        if padH or padW:
            y = y[:, :, :oH, :oW]

        y = self.post(y)
        return x + y  # residual


In [17]:
class BackboneWithWindowAttn(nn.Module):
    def __init__(self, backbone, attn_levels=('3','4'), num_heads=4, window=7, dim_reduction=None):
        super().__init__()
        self.backbone = backbone
        self.attn_levels = set(attn_levels)
        self.attn = nn.ModuleDict()
        self._initialized = False
        self._cfg = dict(num_heads=num_heads, window=window, dim_reduction=dim_reduction)

    @torch.no_grad()
    def _init_for_shapes(self, feats: OrderedDict):
        for k, v in feats.items():
            if k in self.attn_levels:
                C = v.shape[1]
                mod = WindowMHSA(
                    dim=C,
                    num_heads=self._cfg['num_heads'],
                    window_size=self._cfg['window'],
                    dim_reduction=self._cfg['dim_reduction']
                )
                # >>> chave do conserto: mover o módulo pro device do feature map
                mod = mod.to(v.device)
                self.attn[k] = mod
        self._initialized = True

    def forward(self, x):
        feats = self.backbone(x)
        if not self._initialized:
            self._init_for_shapes(feats)
        out = OrderedDict()
        for k, v in feats.items():
            if k in self.attn_levels:
                out[k] = self.attn[k](v)   # agora mesmos devices
            else:
                out[k] = v
        return out


In [29]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


num_classes = 1 + 2

weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
base = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
base.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

base.backbone = BackboneWithWindowAttn(
    base.backbone,
    attn_levels=('3','4'),
    num_heads=4,
    window=7,
    dim_reduction=256
)

model = base.to(device)
model = model.to(device)

imgs0, _ = next(iter(pools_train_loader))
img0 = imgs0[0].to(device)

model.eval()
with torch.no_grad():
    _ = model([img0])

model.train()
for p in model.backbone.backbone.parameters():
    p.requires_grad = False

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=5e-4)


In [30]:
from engine import train_one_epoch, evaluate
for p in model.backbone.backbone.parameters():
    p.requires_grad = False
for name, m in model.backbone.named_modules():
    pass

for p in model.roi_heads.parameters():
    p.requires_grad = True
for p in model.rpn.parameters():
    p.requires_grad = True

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 12
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    if epoch == 2:
        for p in model.backbone.backbone.parameters():
            p.requires_grad = True
        new_params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(new_params, lr=0.002, momentum=0.9, weight_decay=5e-4)

    train_one_epoch(model, optimizer, pools_train_loader, device, epoch, print_freq=10)
    scheduler.step()


Epoch: [0]  [ 0/63]  eta: 0:00:39  lr: 0.000086  loss: 5.8076 (5.8076)  loss_classifier: 1.2038 (1.2038)  loss_box_reg: 0.0885 (0.0885)  loss_objectness: 3.7000 (3.7000)  loss_rpn_box_reg: 0.8154 (0.8154)  time: 0.6283  data: 0.4703  max mem: 4685
Epoch: [0]  [10/63]  eta: 0:00:10  lr: 0.000891  loss: 4.6928 (4.4711)  loss_classifier: 0.9894 (0.8926)  loss_box_reg: 0.1108 (0.1163)  loss_objectness: 3.0217 (2.8457)  loss_rpn_box_reg: 0.6030 (0.6165)  time: 0.2013  data: 0.0432  max mem: 4685
Epoch: [0]  [20/63]  eta: 0:00:07  lr: 0.001697  loss: 2.9258 (3.2484)  loss_classifier: 0.4679 (0.6572)  loss_box_reg: 0.1252 (0.1548)  loss_objectness: 1.9570 (1.9354)  loss_rpn_box_reg: 0.5054 (0.5011)  time: 0.1577  data: 0.0004  max mem: 4685
Epoch: [0]  [30/63]  eta: 0:00:05  lr: 0.002502  loss: 1.1675 (2.6047)  loss_classifier: 0.2668 (0.5393)  loss_box_reg: 0.1971 (0.1844)  loss_objectness: 0.3982 (1.4610)  loss_rpn_box_reg: 0.3073 (0.4201)  time: 0.1569  data: 0.0003  max mem: 4685
Epoch: [

## Treina modelo

In [27]:
from torch.cuda.amp import GradScaler
scaler = GradScaler()

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.002,
    momentum=0.9,
    weight_decay=5e-4
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=10,
    eta_min=1e-5
)



/tmp/ipython-input-7093699.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [32]:
checkpoint_path = '/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/pools/model/bh_pool_withtranformer_c_23.pth'
torch.save(model.state_dict(), checkpoint_path)

## Avalia Modelo

In [31]:
from my_utils.my_utils import evaluate_ap_multi_iou
import numpy as np

csv_path = "/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/pools/bh_results_with_transformer.csv"
df_results = evaluate_ap_multi_iou(
    model=model,
    loader=pools_test_loader,
    device=device,
    score_thr=0.25,
    iou_list=(0.50, 0.75, 0.90),
    iou_grid=np.arange(0.50, 0.96, 0.05),
    out_csv_path=csv_path
)

==== Resultados por classe ====
Classe 1 | mAP[.50:.95]=0.508 | AP@0.50=0.822 | AP@0.75=0.579 | AP@0.90=0.039 (GT=1109)

📊 CSV salvo em: /content/drive/MyDrive/TCC/notebooks/Faster R-CNN/pools/bh_results_with_transformer.csv


In [34]:
from my_utils.my_utils import show_predictions

show_predictions(
    model=model,
    loader=pools_test_loader,
    device=device,
    n=2,
    score_thr=0.25
)


Output hidden; open in https://colab.research.google.com to view.